# Energieverbruik en -productie van Nederland

Energie is duur. Dat weet iedereen. Tijdens dit project gaan "de Potter Robin" & "Verhaegen Stijn" een analyse doen over het energieverbruik en de productie ervan in Nederland.

## Inhoud analyse

1. `Lijst inhoud analyse`

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.rdd import RDD
import matplotlib.pyplot as plt
from pyspark.sql.types import *
from pyspark.sql.functions import *
import glob
import os

In [4]:
spark = SparkSession.builder.appName("EnergyConsumption").getOrCreate()
sc = spark.sparkContext

21/12/01 18:04:06 WARN Utils: Your hostname, Stijns-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.0.120 instead (on interface en0)
21/12/01 18:04:06 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/12/01 18:04:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


Path naar de data

In [5]:
path_gas = r"./data/Gas/"
path_electricity = r"./data/Electricity/"

Het schema dat we zullen gebruiken voor het dataframe van gas en elektriciteit. 

In [6]:
schema = StructType([
    StructField('net_manager', StringType(), True),
    StructField('purchase_area', StringType(), True),
    StructField('street', StringType(), True),
    StructField('zipcode_from', StringType(), True),
    StructField('zipcode_to', StringType(), True),
    StructField('city', StringType(), True),
    StructField('num_connections', DoubleType(), True),
    StructField('delivery_perc', DoubleType(), True),
    StructField('perc_of_active_connections', DoubleType(), True),
    StructField('type_conn_perc', DoubleType(), True),
    StructField('type_of_connection', StringType(), True),
    StructField('annual_consume', DoubleType(), True),
    StructField('year', IntegerType(), True)
  ])

Hier declareren we de make_df functie die de raw data zal omzetten naar een DataFrame. We verwijderen ook een aantal files omdat de data van '*coteq_gas*' hetzelfde is van 2013 - 2018. Ook hebben wij '*rendo_gas_2019*' verwijderd aangezien die null values bezit.

In [7]:
def make_df(path):
    rdd_full = sc.emptyRDD()
    df_full = spark.createDataFrame(rdd_full,schema)
    for filename in os.listdir(path):
        if(filename == "rendo_gas_2019.csv" or 
           filename == "coteq_gas_2014.csv" or 
           filename == "coteq_gas_2015.csv" or 
           filename == "coteq_gas_2016.csv" or 
           filename == "coteq_gas_2017.csv" or 
           filename == "coteq_gas_2018.csv"):
#            filename == "endurisgas_01012015.csv"
            print("removed " + filename)
        else:
            # Read file and assign data
            year = filename[-8:]
            year = year[0:4]
            data = spark.read.option("header","true").option("inferSchema", "true").csv(path + filename, sep=",")
            listColumns = data.columns
            if 'ï»¿NETBEHEERDER' in listColumns:
                data = data.withColumnRenamed("ï»¿NETBEHEERDER","net_manager")
            data = data.select(data.net_manager, data.purchase_area, data.street, data.zipcode_from,
                               data.zipcode_to,
                               data.city,
                               data.num_connections,
                               data.delivery_perc,
                               data.perc_of_active_connections,
                               data.type_conn_perc,
                               data.type_of_connection,
                               data.annual_consume)
            data = data.withColumn('year', lit(int(year)))
            df_full = df_full.union(data)
    return df_full

We gebruiken de functie make_df(path) die we juist gedefinieerd hebben om al de gasdata in te lezen. 

In [6]:
df_gas = make_df(path_gas)


removed coteq_gas_2018.csv
removed rendo_gas_2019.csv
removed coteq_gas_2015.csv
removed coteq_gas_2014.csv
removed coteq_gas_2016.csv
removed coteq_gas_2017.csv


In [7]:
df_gas.show(vertical=True)

-RECORD 0------------------------------------------
 net_manager                | 8717127750009        
 purchase_area              | GAS Gastransport ... 
 street                     | Poststraat           
 zipcode_from               | 4301AA               
 zipcode_to                 | 4301AA               
 city                       | ZIERIKZEE            
 num_connections            | 14.0                 
 delivery_perc              | 100.0                
 perc_of_active_connections | 100.0                
 type_conn_perc             | 50,00                
 type_of_connection         | G4                   
 annual_consume             | 3736.0               
 year                       | 2015                 
-RECORD 1------------------------------------------
 net_manager                | 8717127750009        
 purchase_area              | GAS Gastransport ... 
 street                     | Poststraat           
 zipcode_from               | 4301AB               
 zipcode_to 

In [8]:
df_gas.count()

3665480

In [9]:
df_gas.printSchema()

root
 |-- net_manager: string (nullable = true)
 |-- purchase_area: string (nullable = true)
 |-- street: string (nullable = true)
 |-- zipcode_from: string (nullable = true)
 |-- zipcode_to: string (nullable = true)
 |-- city: string (nullable = true)
 |-- num_connections: double (nullable = true)
 |-- delivery_perc: double (nullable = true)
 |-- perc_of_active_connections: double (nullable = true)
 |-- type_conn_perc: string (nullable = true)
 |-- type_of_connection: string (nullable = true)
 |-- annual_consume: double (nullable = true)
 |-- year: integer (nullable = true)



Hoewel we in het schema meegeven dat de kolom 'type_conn_perc' het datatype Double heeft, is dit nog niet het geval. Het datatype is String. We denken dat dit komt omdat de values komma's bevatten. Bv: 50,0 ipv 50.0. We veranderen dus alle komma's met punten voor deze kolom. Als we dit niet doen en we casten deze kolom meteen naar een double, dan worden de waardes die een komma bevatten omgezet naar null. 

In [10]:
df_gas = df_gas.withColumn('type_conn_perc', regexp_replace('type_conn_perc', '[,]', '.'))

In [11]:
df_gas.show(vertical=True)

-RECORD 0------------------------------------------
 net_manager                | 8717127750009        
 purchase_area              | GAS Gastransport ... 
 street                     | Poststraat           
 zipcode_from               | 4301AA               
 zipcode_to                 | 4301AA               
 city                       | ZIERIKZEE            
 num_connections            | 14.0                 
 delivery_perc              | 100.0                
 perc_of_active_connections | 100.0                
 type_conn_perc             | 50.00                
 type_of_connection         | G4                   
 annual_consume             | 3736.0               
 year                       | 2015                 
-RECORD 1------------------------------------------
 net_manager                | 8717127750009        
 purchase_area              | GAS Gastransport ... 
 street                     | Poststraat           
 zipcode_from               | 4301AB               
 zipcode_to 

Nu we dit gedaan hebben moeten we deze kolom nog casten naar het datatype Double. 

In [12]:
df_gas = df_gas.withColumn('type_conn_perc', df_gas['type_conn_perc'].cast(DoubleType()))

In [13]:
df_gas.printSchema()

root
 |-- net_manager: string (nullable = true)
 |-- purchase_area: string (nullable = true)
 |-- street: string (nullable = true)
 |-- zipcode_from: string (nullable = true)
 |-- zipcode_to: string (nullable = true)
 |-- city: string (nullable = true)
 |-- num_connections: double (nullable = true)
 |-- delivery_perc: double (nullable = true)
 |-- perc_of_active_connections: double (nullable = true)
 |-- type_conn_perc: double (nullable = true)
 |-- type_of_connection: string (nullable = true)
 |-- annual_consume: double (nullable = true)
 |-- year: integer (nullable = true)



Het schema van het dataframe df_gas is nu in orde. Elke kolom heeft het juiste datatype. 

We doorlopen nu hetzelfde proces voor de elektriciteitsdata. We gebruiken de functie make_df(path) om al de elektriciteitsdata in te lezen.

In [8]:
df_electricity = make_df(path_electricity)

In [15]:
df_electricity.show(vertical=True)

-RECORD 0-----------------------------------------
 net_manager                | 8716874000009       
 purchase_area              | Stedin Utrecht      
 street                     | Egelshoek           
 zipcode_from               | 1213RC              
 zipcode_to                 | 1231AB              
 city                       | HILVERSUM           
 num_connections            | 30.0                
 delivery_perc              | 90.0                
 perc_of_active_connections | 100.0               
 type_conn_perc             | 80.0                
 type_of_connection         | 3x25                
 annual_consume             | 5909.0              
 year                       | 2019                
-RECORD 1-----------------------------------------
 net_manager                | 8716874000009       
 purchase_area              | Stedin Utrecht      
 street                     | Rembrandtlaan       
 zipcode_from               | 1231AC              
 zipcode_to                 | 1

In [16]:
df_electricity.count()

4077352

In [17]:
df_electricity.printSchema()

root
 |-- net_manager: string (nullable = true)
 |-- purchase_area: string (nullable = true)
 |-- street: string (nullable = true)
 |-- zipcode_from: string (nullable = true)
 |-- zipcode_to: string (nullable = true)
 |-- city: string (nullable = true)
 |-- num_connections: double (nullable = true)
 |-- delivery_perc: string (nullable = true)
 |-- perc_of_active_connections: string (nullable = true)
 |-- type_conn_perc: string (nullable = true)
 |-- type_of_connection: string (nullable = true)
 |-- annual_consume: double (nullable = true)
 |-- year: integer (nullable = true)



Ook bij het dataframe df_electricity is het schema nog niet correct toegepast. Niet alle kolommen hebben het juiste datatype. Ook hier casten we de kolommen 'delivery_perc', 'perc_of_active_connections' en 'type_conn_perc' naar het datatype dat we in het schema hebben vastgelegd, namelijk een Double. 

In [9]:
df_electricity = df_electricity.withColumn('delivery_perc', df_electricity['delivery_perc'].cast(DoubleType()))
df_electricity = df_electricity.withColumn('perc_of_active_connections', df_electricity['perc_of_active_connections'].cast(DoubleType()))
df_electricity = df_electricity.withColumn('type_conn_perc', df_electricity['type_conn_perc'].cast(DoubleType()))

In [19]:
df_electricity.printSchema()

root
 |-- net_manager: string (nullable = true)
 |-- purchase_area: string (nullable = true)
 |-- street: string (nullable = true)
 |-- zipcode_from: string (nullable = true)
 |-- zipcode_to: string (nullable = true)
 |-- city: string (nullable = true)
 |-- num_connections: double (nullable = true)
 |-- delivery_perc: double (nullable = true)
 |-- perc_of_active_connections: double (nullable = true)
 |-- type_conn_perc: double (nullable = true)
 |-- type_of_connection: string (nullable = true)
 |-- annual_consume: double (nullable = true)
 |-- year: integer (nullable = true)



Het schema van de dataframe df_electricity komt nu overeen met het schema dat we hebben opgesteld. De data van het gas- en elektriciteitsverbruik is nu correct ingeladen. 

Bij de data ontbreken longitude en latitude coördinaten. Om te kunnen plotten op een map heb je deze waarden nodig.De naam van de stad is wel aanwezig. We zijn dan op zoek gegaan naar data van de steden van Nederland met hun longitude en latitude. We hebben een dataset gevonden hiervoor die de longitude en latitude bevat voor 944 prominente Nederlandse steden. 

Er is vreemd genoeg 1 stad in deze dataset die een Engels naam heeft. Namelijk 'The Hague', die zullen we omzetten naar de Nederlands naam 'Den Haag'.

In [10]:
df_cities = spark.read.option("header","true").option("inferSchema", "true").csv("./data/nl.csv", sep=",")

In [11]:
df_cities = df_cities.withColumn('city', regexp_replace('city', 'The Hague', 'Den Haag'))

In [57]:
df_cities.show(vertical=True)

-RECORD 0--------------------------------
 city              | Den Haag            
 lat               | 52.0767             
 lng               | 4.2986              
 country           | Netherlands         
 iso2              | NL                  
 admin_name        | Zuid-Holland        
 capital           | primary             
 population        | 1406000             
 population_proper | 501725              
-RECORD 1--------------------------------
 city              | Amsterdam           
 lat               | 52.3667             
 lng               | 4.8833              
 country           | Netherlands         
 iso2              | NL                  
 admin_name        | Noord-Holland       
 capital           | primary             
 population        | 862965              
 population_proper | 862965              
-RECORD 2--------------------------------
 city              | Utrecht             
 lat               | 52.0908             
 lng               | 5.1222       

In [22]:
df_cities.printSchema()

root
 |-- city: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- lng: double (nullable = true)
 |-- country: string (nullable = true)
 |-- iso2: string (nullable = true)
 |-- admin_name: string (nullable = true)
 |-- capital: string (nullable = true)
 |-- population: integer (nullable = true)
 |-- population_proper: integer (nullable = true)



In [23]:
print(df_electricity.select(approx_count_distinct("city")).collect()[0][0])

21/12/01 16:10:39 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


2533


Wat we willen laten zien op de map is de hoeveelheid jaarlijks verbruik van elektriciteit per stad. In de originele data zijn er veel meer steden aanwezig dan de data die we hebben kunnen vinden waarbij de longitude en latitude coördinaten zijn. We zullen hierdoor enkel de steden waarvoor we de coördinaten hebben kunnen laten zien. 

In [58]:
print(df_cities.first())

Row(city='Den Haag', lat=52.0767, lng=4.2986, country='Netherlands', iso2='NL', admin_name='Zuid-Holland', capital='primary', population=1406000, population_proper=501725)


In [63]:
import folium
map = folium.Map(location=[52.1326, 5.29131], zoom_start=6)

for city in df_cities.rdd.collect():
    folium.CircleMarker(location=[city.lat, city.lng]).add_to(map)
map

Voor het jaar 2019, verbruik elektriciteit per stad op map laten zien. 

In [12]:
df_electricity = df_electricity.withColumn('city', lower(col('city')))

In [13]:
df_cities = df_cities.withColumn('city', lower(col('city')))

In [67]:
df_electricity_2019 = df_electricity.filter(df_electricity.year == 2019)

In [68]:
df_electricity_2019.count()

366069

In [31]:
print(df_electricity_2019.filter(df_electricity_2019.city == 'hilversum').first().annual_consume)

5909.0


In [17]:
list_cities_with_lat_long = df_cities.select('city').rdd.flatMap(lambda x: x).collect()
print(list_cities_with_lat_long)

['den haag', 'amsterdam', 'utrecht', 'rotterdam', 'eindhoven', 'almere', 'groningen', 'breda', 'apeldoorn', 'haarlem', 'zaanstad', 'arnhem', '’s-hertogenbosch', 'leeuwarden', 'maastricht', 'zwolle', 'alphen aan den rijn', 'emmen', 'delft', 'deventer', 'hilversum', 'heerlen', 'purmerend', 'lelystad', 'roosendaal', 'spijkenisse', 'ede', 'gouda', 'zaandam', 'bergen op zoom', 'capelle aan den ijssel', 'veenendaal', 'katwijk', 'zeist', 'nieuwegein', 'assen', 'hardenberg', 'barneveld', 'roermond', 'heerhugowaard', 'oosterhout', 'den helder', 'hoogeveen', 'kampen', 'woerden', 'houten', 'sittard', 'ijmuiden', 'middelburg', 'harderwijk', 'zutphen', 'ridderkerk', 'kerkrade', 'veldhoven', 'medemblik', 'zwijndrecht', 'vlissingen', 'rheden', 'etten-leur', 'zevenaar', 'venray', 'noordwijk', 'tiel', 'uden', 'huizen', 'beverwijk', 'wijchen', 'dronten', 'hellevoetsluis', 'maarssen', 'leidschendam', 'heemskerk', 'veghel', 'goes', 'venlo', 'landgraaf', 'teijlingen', 'geleen', 'hellendoorn', 'castricum', 

In [18]:
df_electricity_filtered = df_electricity.filter(df_electricity.city.isin(list_cities_with_lat_long))


In [71]:
df_electricity_filtered.first()

21/12/01 17:53:55 WARN DAGScheduler: Broadcasting large task binary with size 1182.2 KiB


Row(net_manager='8716874000009', purchase_area='Stedin Utrecht', street='Egelshoek', zipcode_from='1213RC', zipcode_to='1231AB', city='hilversum', num_connections=30.0, delivery_perc=90.0, perc_of_active_connections=100.0, type_conn_perc=80.0, type_of_connection='3x25', annual_consume=5909.0, year=2019)

In [19]:
df_electricity_filtered.count()

3080965

In [36]:
print(df_electricity_filtered.first().city)

hilversum


21/12/01 16:11:13 WARN DAGScheduler: Broadcasting large task binary with size 1182.2 KiB


In [80]:
print(df_cities.first().city)

den haag


In [39]:
df_electricity_filtered.select(countDistinct("city")).show()

21/12/01 16:21:36 WARN DAGScheduler: Broadcasting large task binary with size 1804.5 KiB
21/12/01 16:21:44 WARN DAGScheduler: Broadcasting large task binary with size 1255.5 KiB


+--------------------+
|count(DISTINCT city)|
+--------------------+
|                 811|
+--------------------+



In [40]:
df_electricity.select(countDistinct("city")).show()

+--------------------+
|count(DISTINCT city)|
+--------------------+
|                2624|
+--------------------+



In [ ]:
df_electricity_filtered.show()

In [54]:
for x in df_electricity_filtered.take(5):
    city = df_cities.filter(df_cities.name == x.city)
    print(city.first().name)
    print(x)

21/12/01 16:55:12 WARN DAGScheduler: Broadcasting large task binary with size 1182.2 KiB


hilversum
Row(net_manager='8716874000009', purchase_area='Stedin Utrecht', street='Egelshoek', zipcode_from='1213RC', zipcode_to='1231AB', city='hilversum', num_connections=30.0, delivery_perc=90.0, perc_of_active_connections=100.0, type_conn_perc=80.0, type_of_connection='3x25', annual_consume=5909.0, year=2019)
kortenhoef
Row(net_manager='8716874000009', purchase_area='Stedin Utrecht', street='Vreelandseweg', zipcode_from='1241EZ', zipcode_to='1391AB', city='kortenhoef', num_connections=63.0, delivery_perc=98.41, perc_of_active_connections=100.0, type_conn_perc=71.0, type_of_connection='3x25', annual_consume=7236.0, year=2019)
abcoude
Row(net_manager='8716874000009', purchase_area='Stedin Utrecht', street='Koningsvaren', zipcode_from='1391AC', zipcode_to='1391AC', city='abcoude', num_connections=12.0, delivery_perc=100.0, perc_of_active_connections=100.0, type_conn_perc=92.0, type_of_connection='3x25', annual_consume=5153.0, year=2019)
abcoude
Row(net_manager='8716874000009', purchas

In [20]:
df_electricity_filtered = df_electricity_filtered.join(df_cities, ['city'])

In [21]:
df_electricity_filtered

DataFrame[city: string, net_manager: string, purchase_area: string, street: string, zipcode_from: string, zipcode_to: string, num_connections: double, delivery_perc: double, perc_of_active_connections: double, type_conn_perc: double, type_of_connection: string, annual_consume: double, year: int, lat: double, lng: double, country: string, iso2: string, admin_name: string, capital: string, population: int, population_proper: int]

In [34]:
import folium
map = folium.Map(location=[52.1326, 5.29131], zoom_start=6)

year = 2019
df_electricity_2019 = df_electricity_filtered.filter(df_electricity.year == 2019)
df_electricity_2019 = df_electricity_2019.groupBy("city", "lat", "lng").agg(sum("annual_consume").alias("annual_consume"))

for row in df_electricity_2019.rdd.collect():
    radius = 5
    if row.annual_consume >= 1000000:
        color = 'red'
    elif row.annual_consume >= 500000:
        color = 'orange'
    else:
        color = 'green'
    folium.CircleMarker(location=[row.lat, row.lng], radius=radius, color=color, fill_color=color, fill=True).add_to(map)
map

In [30]:
df_electricity_2019.show()

+------------+-------+------+------------------+
|        city|    lat|   lng|    annual_consume|
+------------+-------+------+------------------+
| schipluiden|51.9833|4.3167|          473556.0|
|     leerdam|51.8939|5.0914|         1435087.0|
|      deurne|51.4639|5.7947|3243627.2599999984|
|  streefkerk|51.8994|4.7419|          226125.0|
|  hekelingen|51.8267|4.3422|          115140.0|
|   amsterdam|52.3667|4.8833|       5.4836591E7|
|       dalem|51.8278|5.0094|           75231.0|
|monnickendam|52.4547|5.0353|          910052.0|
|    kerkwijk|51.7744|5.2194|           69887.0|
|     huissen|51.9333|5.9333|         1743744.0|
|  den dungen|51.6653|5.3714| 427103.3800000002|
|   wassenaar|52.1453|4.4006|         3461003.0|
|  oostvoorne|51.9119|4.1008|          782592.0|
|   groesbeek|51.7833|5.9333|         1712511.0|
|      goirle|51.5203|5.0671|1815669.8899999976|
|  posterholt|51.1219|6.0378|         378861.71|
|   giethoorn|52.7397|6.0775|         426210.65|
|  kruiningen|51.448

In [31]:
df_electricity_2019.printSchema()

root
 |-- city: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- lng: double (nullable = true)
 |-- annual_consume: double (nullable = true)



## Data over elektriciteitproductie in Nederland. (Elektriciteit__aanbod_en_verbruik_12102021_102309.csv

We hebben eerst meteen een dataframe gemaakt van het csv-bestand door de optie "header" "true" en de optie "inferSchema" "true". Zo wordt het schema van de dataframe zelf bepaalt bij het inlezen. 

Daarna hebben we ook eerst een rdd gemaakt van het csv-bestand om vervolgens zelf het schema te bepalen en toe te passen om zo een dataframe te krijgen.

We hebben beide eens toegepast om goed het verschil tussen een rdd en dataframe te begrijpen. 

In [ ]:
df_electricity_production = spark.read.option("header","true").option("inferSchema", "true").csv("./data/Elektriciteit__aanbod_en_verbruik_12102021_102309.csv", sep=";")
electricity_production_raw = sc.textFile("./data/Elektriciteit__aanbod_en_verbruik_12102021_102309.csv")

df_electricity_production.show(vertical = True)
df_electricity_production.printSchema()

In [ ]:
# Load a text file and convert each line to a Row, so that a DataFrame can be made.

electricity_production_parts = electricity_production_raw.map(lambda l: l.split(";"))
electricity_production_rows = electricity_production_parts.map(lambda l: Row(periode=l[0], netto_productie=l[3],
                                                                             netto_productie_brandstoffen=l[5],
                                                                             netto_productie_windenergie=l[12],
                                                                             netto_productie_zonnestroom=l[15],
                                                                             netto_verbruik=l[30]))

In [ ]:
electricity_production_rows.take(5)


In [ ]:

header = electricity_production_rows.first()
df_electricity_production = spark.createDataFrame(electricity_production_rows.filter(lambda row: row != header))

In [ ]:
df_electricity_production.show()

df2_electricity_production = df_electricity_production.selectExpr("cast(periode as string) periode",
                                                                  "cast(netto_productie as int) netto_productie",
                                                                  "cast(netto_productie_brandstoffen as int) netto_productie_brandstoffen",
                                                                  "cast(netto_productie_windenergie as int) netto_productie_windenergie",
                                                                  "cast(netto_productie_zonnestroom as int) netto_productie_zonnestroom",
                                                                  "cast(netto_verbruik as int) netto_verbruik")
df2_electricity_production.printSchema()

df2_electricity_production = df2_electricity_production.withColumn('periode', regexp_replace('periode', '["*]', ''))
df2_electricity_production.show()

In [ ]:
# Get values from the needed columns in a List to use them to plot.
x_axis = df2_electricity_production.rdd.map(lambda l:l.periode).collect()
y_axis1 = df2_electricity_production.rdd.map(lambda l:l.netto_productie).collect()
y_axis2 = df2_electricity_production.rdd.map(lambda l:l.netto_verbruik).collect()
figure_netto_production = plt.figure(figsize=(7, 7))
figure_netto_production.suptitle('Electriciteitproductie/verbruik NL', fontsize=11)
plt1 = figure_netto_production.add_subplot(111)
plt1.plot(x_axis, y_axis1, label = "netto productie")
plt1.plot(x_axis, y_axis2, label = "netto verbruik")
plt.xlabel("Year")
plt.ylabel("mln kWh")
plt.legend()
plt.show()

In [ ]:
# Lets focus on renewable energy, show how much the share of renewable energy is of the whole netto production
labels = "Brandstoffen","Windenergie","Zonnestroom"
def show_renewable_energy_share(year):
    energy_distribution = df2_electricity_production.filter(df2_electricity_production.periode == year).collect()

    data = [energy_distribution[0].asDict()["netto_productie_brandstoffen"],energy_distribution[0].asDict()["netto_productie_windenergie"],energy_distribution[0].asDict()["netto_productie_zonnestroom"]]
 
    explode = (0, 0.1, 0.1)  # highlight the 2nd and 3rd slice of the pie chart
    fig1, ax1 = plt.subplots()
    plt.title('Energieproductie ' + str(year))
    ax1.pie(data, explode=explode, labels=labels, autopct='%1.1f%%',
    shadow=True, startangle=90)
    ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
    plt.show()

In [ ]:
from IPython.display import HTML,display

form = """
<style>
    .ep-container {
        background-color: #f8f9fa; 
        width:300px; 
        padding:20px; 
        color: #495057;
    }
    .ep-button {
        padding:5px; 
        width: 100px; 
        border: none; 
        background-color: #ced4da; 
        color: #495057;
        transition: ease-in-out 0.1s;
    }
    .ep-button:hover {
        background-color: #adb5bd; 
        color: #343a40;
    }
    .ep-form-inputs-control {
        display: flex;
        flex-wrap: wrap;
        margin-top: 1em;
        margin-bottom: 2em;
    }
    .ep-input-wrapper {
        display: flex;
        margin-left: 1em;
        margin-right: 1em;
    }
</style>
<div class="ep-container">
<p> Select a year and press the Submit button to see how much the share of renewable energy is of the entire electricity production for that year:<p>
<div class="ep-form-inputs-control">
    <div class="ep-input-wrapper"><input type="radio" name="year" value="2015"><label>2015</label></div>
    <div class="ep-input-wrapper"><input type="radio" name="year" value="2016"><label>2016</label></div>
    <div class="ep-input-wrapper"><input type="radio" name="year" value="2017"><label>2017</label></div>
    <div class="ep-input-wrapper"><input type="radio" name="year" value="2018"><label>2018</label></div>
    <div class="ep-input-wrapper"><input type="radio" name="year" value="2019"><label>2019</label></div>
    <div class="ep-input-wrapper"><input type="radio" name="year" value="2020"><label>2020</label></div>
</div>
<button class="ep-button" type="button" onclick="set_vars()">Submit</button>
</div>
"""
javascript = """
<script type="text/Javascript">
    function set_vars(){
        var var_name1 = 'inp_year';
        var ele = document.getElementsByName('year');
        
        for(i = 0; i < ele.length; i++) {
                if(ele[i].checked)
                var var_value1 = ele[i].value;
            }
        var command1 = var_name1 + " = '" + var_value1 + "'";
        var kernel = IPython.notebook.kernel;
        kernel.execute(command1);
        cell = IPython.notebook.get_cell(40);
        cell.execute();
        cell.focus_cell(40);
    }
</script>
"""

HTML(form + javascript)

In [ ]:
print("Year: " + inp_year)
show_renewable_energy_share(int(inp_year))

Als je de pie chart bekijkt van de verschillende jaren dan zie je dat er toch meer en meer ingezet wordt op hernieuwbare energie en dat de energieproductie door middel van het verbranden van brandstoffen ieder jaar afneemt. 

In [ ]:

## Belangrijk!! vergeet cellnumber niet aan te passen bij de interactieve widget.
## Al geprobeerd met get_selected_cell() maar heb het nog niet werkend gekregen. 